In [1]:
import pandas

In [7]:
data = pandas.read_csv('test.csv', sep = '\t')

In [8]:
data

,Python Raw Time,Python Raw Energy (Joules),IO500 Raw Time,IO500 Raw Energy (Joules),TPCC Raw Time,TPCC Raw Energy (Joules),Noop Raw Time,Noop Raw Energy (Joules),Time (Minute),Python Energy per Minute,IO500 Energy per Minute,TPCC Energy per Minute,Noop Energy per Minute
0,12:47:46,1.11,17:40:31,0.77,20:09:42,0.89,12:30:54,1.07,0.0,174.32,207.98,215.22,4.86
1,12:47:47,0.92,17:40:32,0.48,20:09:43,0.50,12:30:55,0.63,1.0,1107.02,463.90,777.18,46.96
2,12:47:48,13.52,17:40:33,10.49,20:09:44,15.61,12:30:56,0.71,2.0,1019.38,461.04,688.59,28.07
3,12:47:49,11.34,17:40:34,15.71,20:09:45,15.70,12:30:57,0.43,3.0,1039.86,419.70,408.29,27.04
4,12:47:50,11.78,17:40:35,18.54,20:09:46,8.61,12:30:58,0.59,4.0,1072.29,449.95,539.91,29.06
...,...,...,...,...,...,...,...,...,...,...,...,...,...
12474,NaN,NaN,NaN,NaN,23:37:38,6.79,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12475,NaN,NaN,NaN,NaN,23:37:39,5.10,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12476,NaN,NaN,NaN,NaN,23:37:40,5.12,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12477,NaN,NaN,NaN,NaN,NaN,0.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN
